<a href="https://colab.research.google.com/github/rafserran0/ProjetoGeminiStudio/blob/main/templates/aistudio_gemini_prompt_chat_b64.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [25]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import


In [79]:
!pip install -U -q google-generativeai

In [81]:
# import necessary modules.
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown
from google.colab import userdata
from google.colab import files

## Set the API key

Add your API_KEY to the secrets manager in the left pannel "🔑".

In [97]:
# Configure the client library by providing your API key.
genai.configure(api_key=userdata.get('API'))

### Parse the arguments

In [87]:
model = 'gemini-1.5-pro-latest' # @param {isTemplate: true}
contents_b64 = '...' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjowLCJtYXhfb3V0cHV0X3Rva2VucyI6ODE5Miwic3RvcF9zZXF1ZW5jZXMiOltdfQ==' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfSEFURV9TUEVFQ0giLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfU0VYVUFMTFlfRVhQTElDSVQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfREFOR0VST1VTX0NPTlRFTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn1d' # @param {isTemplate: true}
user_input_b64 = '' # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))
user_input = base64.b64decode(user_input_b64).decode()
stream = False

In [88]:
contents

[{'role': 'user',
  'parts': 'Sei que a google tem uma parceria com a Automation Anywhere para uso de automações de processos mais inteligentes, incluindo ia generativa e leitura de documentos porém como isso funciona na prática ?'},
 {'role': 'model',
  'parts': '## Google Cloud e Automation Anywhere: Uma parceria para automações mais inteligentes\n\nA parceria entre Google Cloud e Automation Anywhere visa, de fato, tornar as automações de processos mais inteligentes e eficientes, através da integração de tecnologias como IA generativa e leitura de documentos. Mas como isso funciona na prática? Vamos explorar alguns pontos chave:\n\n**1. Document AI e IQ Bot:**\n\n*   **Extração inteligente de dados:** O Document AI do Google Cloud utiliza machine learning para extrair informações relevantes de documentos, como formulários, contratos e faturas. Essa capacidade se integra ao IQ Bot da Automation Anywhere, permitindo que bots compreendam e processem dados de documentos de forma mais efi

In [66]:
generation_config

{'temperature': 1,
 'top_p': 0.95,
 'top_k': 0,
 'max_output_tokens': 8192,
 'stop_sequences': []}

In [89]:
safety_settings

[{'category': 'HARM_CATEGORY_HARASSMENT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_HATE_SPEECH',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'}]

In [90]:
user_input="Sugerir então uma tabela completa de análise de complexidade de um RPA ponderando os pesos de cada um dos critérios mais importantes e uma classificação em 5 faixas, junto com o esforço em horas para cada nivel e explicação de como fazer o cálculo"

### Call the API

In [100]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

chat = gemini.start_chat(history=contents)

response = chat.send_message(
    user_input,
    stream=stream)

In [101]:
display(Markdown(response.text))

## Tabela de Análise de Complexidade de RPA com Esforço Estimado

Essa tabela oferece uma estrutura para avaliar a complexidade de um projeto de RPA, ponderando os critérios mais importantes e classificando-o em 5 faixas de complexidade. Além disso, apresenta uma estimativa de esforço em horas para cada nível, auxiliando no planejamento e gestão de projetos de automação.

**Tabela:**

| Critério | Peso | Baixa (1) | Média-Baixa (2) | Média (3) | Média-Alta (4) | Alta (5) |
|---|---|---|---|---|---|---| 
| Número de Aplicações | 5 | 1 aplicação | 2-3 aplicações | 4-5 aplicações | 6-8 aplicações | Mais de 8 aplicações |
| Tipo de Aplicações | 4 | Web com interfaces simples | Misto de web e desktop, interfaces padrão | Misto de web, desktop, legadas, interfaces complexas | Legadas, mainframe, Citrix, interfaces complexas/instáveis | Diversas tecnologias, interfaces complexas, legados e cloud |
| Estrutura dos Dados | 3 | Estruturados (bancos de dados, planilhas) | Semi-estruturados (CSV, XML) | Não estruturados (emails, PDFs) | Misto de estruturados, semi-estruturados e não estruturados | Altamente não estruturados e dinâmicos |
| Lógica do Processo | 5 | Linear, sem regras complexas | Regras de negócio, decisão simples | Lógica complexa, múltiplas regras, decisão condicional | Lógica complexa, decisão baseada em dados externos/ML | Alta variabilidade, imprevisibilidade, IA/ML avançado | 
| Exceções e Erros | 4 | Poucas, fáceis de tratar | Moderadas, regras de tratamento definidas | Frequentes, complexas, decisão para tratamento | Frequentes, imprevisíveis, tratamento avançado | Grande volume, complexas, aprendizado de máquina |

**Pontuação:**

* Some os pontos de cada critério multiplicado pelo seu peso. 
    * Exemplo: Se "Número de Aplicações" recebe pontuação 3 (4-5 aplicações), a pontuação ponderada é 3 * 5 (peso) = 15 pontos.
* O total de pontos determina a faixa de complexidade.

**Faixas de Complexidade e Esforço Estimado:**

| Pontuação | Classificação | Esforço (horas) |
|---|---|---|
| 0-50 | Baixa | 40-80 |
| 51-100 | Média-Baixa | 80-160 |
| 101-150 | Média | 160-320 |
| 151-200 | Média-Alta | 320-640 |
| 201-250 | Alta | 640+ |

**Cálculo do Esforço:**

O esforço em horas é uma estimativa e pode variar significativamente dependendo de fatores como:

* Experiência da equipe de desenvolvimento
* Qualidade da documentação do processo
* Disponibilidade de recursos e infraestrutura
* Complexidade técnica das integrações

A tabela fornece uma base para estimativa, mas ajuste-a de acordo com a realidade do seu projeto.

**Exemplo de Aplicação:**

Vamos imaginar um processo de RPA para automatizar o cadastro de clientes em um sistema CRM. 

* Número de Aplicações: 3 (site, planilha Excel, CRM) - Pontuação: 2, Ponderada: 10
* Tipo de Aplicações: Misto web e desktop, interfaces padrão - Pontuação: 2, Ponderada: 8
* Estrutura dos Dados: Semi-estruturados (planilha Excel) - Pontuação: 2, Ponderada: 6
* Lógica do Processo:  Regras de negócio, decisão simples - Pontuação: 2, Ponderada: 10
* Exceções e Erros: Moderadas, regras de tratamento definidas - Pontuação: 2, Ponderada: 8

Total de Pontos: 42 - Classificação: **Média-Baixa**, Estimativa de Esforço: **80-160 horas**.

**Observações:**

* A tabela e a estimativa de esforço são ferramentas para auxiliar na análise e planejamento.
* Realize uma análise mais detalhada considerando as características específicas de cada projeto. 
* Ajuste os pesos dos critérios e as faixas de esforço conforme a sua experiência e contexto.

Lembre-se: Essa metodologia visa simplificar a avaliação da complexidade de um RPA. O ideal é que a análise seja complementada por uma avaliação mais aprofundada de cada critério, utilizando a tabela como guia.

In [ ]:
response.prompt_feedback

In [ ]:
response.candidates